# Evaluation Chain

This chain classifies user's input as a critiera type defined in `CriteriaEnum`. 

Based on the classified type, the chain will evaluate whether this input has met the criteria description.

Note: This might be swapped out with a rule based approach.

In [1]:
import sys
!{sys.executable} -m pip install langchain-community langchain langchain-core google-cloud-aiplatform google-cloud-discoveryengine

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [4]:
CRITERIA_TEMPLATE = """
Respond Y or N based on how well the following response follows the specified rubric. Grade only based on the rubric and expected response:

Grading Rubric: {criteria}

DATA:
---------
Question: {question}
---------
{format_instructions}
"""

CLASSIFIER_TEMPLATE = """
Given the user question below, classify it as either being about one of the criteria's in this list
{criteria_names}

Do not respond with more than one word. 

<question>
{question}
</question>

Classification:
"""


In [5]:
# ! ls -la /opt/conda/lib/python3.10/site-packages/langchain_core/output_parsers/__init__.py
# ! cat /opt/conda/lib/python3.10/site-packages/langchain_core/output_parsers/__init__.py

! ls -la /opt/conda/lib/python3.10/site-packages/ | grep langchain

from langchain_community.llms import VertexAI


drwxr-xr-x  33 jupyter jupyter      4096 Feb 28 13:31 langchain
drwxr-xr-x   2 jupyter jupyter      4096 Feb 28 13:31 langchain-0.1.0.dist-info
drwxr-xr-x  22 jupyter jupyter      4096 Feb 28 13:31 langchain_community
drwxr-xr-x   2 jupyter jupyter      4096 Feb 28 13:31 langchain_community-0.0.9.dist-info
drwxr-xr-x  19 jupyter jupyter      4096 Feb 28 13:31 langchain_core
drwxr-xr-x   2 jupyter jupyter      4096 Feb 28 13:31 langchain_core-0.1.7.dist-info


In [6]:
from enum import Enum


class CriteriaEnum(str, Enum):
    """The types of the Criteria."""
    LOW_INCOME = "Is the user or the person in question considered low income?"
    LOW_FINANCIAL_AND_OR_LEGAL_LITERACY = "Does the person in question demonstrate a limited understanding of fundamental financial and/or legal concepts?"
    FAMILY_OF_PWD = "Is the person in question a family of a person with disability?"
    ACCIDENT = "Was the user or person in question recently in an accident?"


In [7]:
from operator import itemgetter
from typing import Dict

from langchain_community.llms import VertexAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch, RunnableSequence, RunnableParallel
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate


class Answer(BaseModel):
    """
    Typings for Evaluator output
    """
    criteria: str = Field(description="criteria")
    reasoning: str = Field(description="detailed explanation")
    value: str = Field(description="Y/N")


class EvaluationChain():
    """This class evaluates the user's input"""

    def __init__(self, eligibility_dict: Dict[str, bool]) -> None:
        """This method instantiates an instance of EvaluationChain"""
        # pylint: disable-next=not-callable
        self.model = VertexAI(
            model="chat-bison-32k", temperature=0, verbose=True, max_tokens=32768)
        self.eligibility_dict = eligibility_dict
        self.chain = self.get_chain()

    def get_chain(self) -> RunnableSequence:
        """
        This method returns the evaluation chain.

        ..evaluation_chain
        classifier_chain | RunnableBranch

        The classifier_chain classifies the input as one of the types in CriteriaEnum
        and passes it to RunnableBranch. The matched runnable is then invoked based
        on the value from the classifier_chain.

        ..RunnableBranch consists of
            1. Default Runnable that returns None if no branches are matched
            2. Custom runnables loaded with each criteria in CriteriaEnum
        """
        evaluators = self.get_evaluators()
        classifier_chain = self.get_classifier_chain()

        def update_and_invoke(evaluator: RunnableSequence, question: str, criteria: str) -> Dict[str, str]:
            """
            This helper function does:
            1. Invokes evaluation chain
            2. Updates eligibility criteria dictionary based on the outcome in 1
            """
            res = evaluator.invoke(question)

            if res["value"] == "Y" and self.eligibility_dict[criteria] in [False, None]:
                self.eligibility_dict[criteria] = True
            return res

        branch = RunnableBranch(
            *[
                (
                    lambda x, k=k: k in x["criteria"].strip() and self.eligibility_dict[k] in [
                        False, None],
                    lambda x, k=k, v=v: update_and_invoke(v, x["question"], k)
                ) for k, v in evaluators.items()
            ],
            lambda x: None
        )
        eval_chain = (
            RunnableParallel({
                "criteria": classifier_chain,
                "question": lambda x: x["question"]
            })
            | branch
        )

        return eval_chain

    def get_classifier_chain(self) -> RunnableSequence:
        """
        This method returns a classifier chain that classifies the question as one
        of the criteria defined in CriteriaEnum.

        If no criteria is matched, None is returned.
        """
        criteria_names = [criteria.name for criteria in CriteriaEnum]
        CLASSIFIER_PROMPT = PromptTemplate.from_template(CLASSIFIER_TEMPLATE)
        classifier_chain = (
            {
                "question": RunnablePassthrough() | itemgetter("question"),
                "criteria_names": lambda x: criteria_names
            }
            | CLASSIFIER_PROMPT
            # pylint: disable-next=not-callable
            | VertexAI(verbose=True)
            | StrOutputParser()
        )
        return classifier_chain

    def get_evaluators(self) -> Dict[str, RunnableSequence]:
        """
        This method returns a dictionary of CRITERIA_NAME : CRITERIA_CHAIN
        """
        parser = JsonOutputParser(pydantic_object=Answer)
        CRITERIA_PROMPT = PromptTemplate.from_template(
            CRITERIA_TEMPLATE, partial_variables={"format_instructions": parser.get_format_instructions()}
        )
        return {
            criteria.name: (
                {
                    "question": RunnablePassthrough(),
                    "criteria": lambda x, c=criteria: {c.name: c.value}
                }
                | CRITERIA_PROMPT
                # pylint: disable-next=not-callable
                | VertexAI(verbose=True)
                | parser
            ) for criteria in CriteriaEnum
        }


In [8]:
eligibility_dict = {
            criteria.name: None
            for criteria in CriteriaEnum
        }

eval_chain = EvaluationChain(eligibility_dict).chain

/opt/conda/envs/python3-11-6/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [9]:
query = "My grandma just had an accident. I don't know what to do."

res = eval_chain.invoke({ "question": query })

print(res)
print(eligibility_dict)

{'criteria': 'ACCIDENT', 'reasoning': 'The user mentioned that their grandma just had an accident.', 'value': 'Y'}
{'LOW_INCOME': None, 'LOW_FINANCIAL_AND_OR_LEGAL_LITERACY': None, 'FAMILY_OF_PWD': None, 'ACCIDENT': True}


In [10]:
query = "My grandma is in the hospital but I don't know enough about financial stuff to know if insurance can cover her bills."

res = eval_chain.invoke({ "question": query })

print(res)
print(eligibility_dict)

{'criteria': 'LOW_FINANCIAL_AND_OR_LEGAL_LITERACY', 'reasoning': 'The person in question acknowledges their lack of knowledge about financial matters, specifically regarding insurance coverage for medical bills.', 'value': 'Y'}
{'LOW_INCOME': None, 'LOW_FINANCIAL_AND_OR_LEGAL_LITERACY': True, 'FAMILY_OF_PWD': None, 'ACCIDENT': True}
